In [6]:
import requests
import pandas as pd
from api_key import alpha_vantage

In [10]:
from api_key import alpha_vantage

stocks = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'NFLX']
sentiment_df = []

for stock in stocks:
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock}&time_from=20150101T0130&apikey={alpha_vantage}'
    response = requests.get(url)

    if response.status_code == 200:
        try:
            news_feed = response.json()['feed']

            stock_sentiments = [{
                'date': item['time_published'][:4]+'-'+item['time_published'][4:6]+'-'+item['time_published'][6:8],
                'ticker': stock,
                'sentiment_score': float(item.get('overall_sentiment_score', 0))
            } for item in news_feed]

            df = pd.DataFrame(stock_sentiments)
            sentiment_df.append(df)
        except KeyError:
            print(f'No sentiment found for {stock}')
    else:
        print(f'API error for {stock}: {response.status_code}')

sentiment_data = pd.concat(sentiment_df)

pivoted_sentiments = sentiment_data.pivot_table(index='date', columns='ticker', values='sentiment_score', aggfunc='mean')
pivoted_sentiments = pivoted_sentiments.fillna(method='ffill').fillna(method='bfill')

pivoted_sentiments.head()

C:\Users\ROG\AppData\Local\Temp\ipykernel_16964\2499348851.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pivoted_sentiments = pivoted_sentiments.fillna(method='ffill').fillna(method='bfill')


ticker,AAPL,AMZN,GOOG,MSFT,NFLX
date,,,,,
2025-04-03,0.108453,0.152113,0.250044,0.086048,0.213562
2025-04-04,0.108453,0.152113,0.250044,0.086048,0.321464
2025-04-05,0.108453,0.152113,0.250044,0.086048,0.215420
2025-04-06,0.108453,0.152113,0.250044,0.086048,0.280937
2025-04-07,0.108453,0.152113,0.250044,0.086048,0.262796


In [11]:
pivoted_sentiments.to_csv('./data/sentiment_data.csv', index=True)